In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
import pickle
import numpy as np

In [3]:
with open("Word_Dict.pkl","rb") as f:
    word_index = pickle.load(f)

with open("One-Hot.pkl","rb") as f:
    One_Hot = pickle.load(f)

In [4]:
One_Hot.shape

(84, 10, 51)

In [5]:
import pandas as pd
df = pd.read_pickle('NLP_df.pkl')
df

,intent,sentence,QorA,NLP
0,인사,안녕하쇼,Q,"[안녕, 하, 쇼]"
1,인사,안녕,Q,[안녕]
2,인사,안녕하세요,Q,[안녕하세요]
3,인사,ㅎㅇ,Q,[ㅎㅇ]
4,인사,ㅎㅇㅇ,Q,[ㅎㅇㅇ]
...,...,...,...,...
79,대건고등학교의 교화,대건고 꽃,Q,"[대, 건고, 꽃]"
80,대건고등학교의 교화,대건고의 꽃은?,Q,"[대, 건고, 의, 꽃, 은, ?]"
81,대건고등학교의 교화,대건고등학교의 꽃은?,Q,"[대건고등학교, 의, 꽃, 은, ?]"
82,대건고등학교의 교화,대건고등학교의 꽃은 무엇이니?,Q,"[대건고등학교, 의, 꽃, 은, 무엇, 이, 니, ?]"


In [6]:
def create_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model


In [7]:
# Assuming your input data shape is (num_samples, time_steps, num_features)
input_shape = (10, 51)  # Adjust accordingly to your data

vocab_size = len(word_index) # Size of the vocabulary (number of unique tokenized morphemes)
embedding_dim = 200  # Dimension of word embeddings
max_sequence_length = 10  # Maximum length of input sequences (after padding/truncating)
num_classes = len(df['intent'].unique())  # Number of complaint categories (number of classes to predict)
hidden_units = 64  # Number of LSTM units (hidden units in the LSTM layer)

# Number of intent classes (6 in this case)
num_classes = 6

model = create_lstm_model(input_shape, num_classes)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [8]:
One_Hot

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [9]:
from sklearn.model_selection import train_test_split


In [10]:
y = []
n = 0
for i in df['intent'].unique() :
    for a in range(len(df[df['intent']==i])) :
        y.append(n)
    n += 1
y = np.array(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(One_Hot, y, test_size=0.2, random_state=42)

In [12]:
# Assuming your data is stored in X_train (input data) and y_train (target labels)
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10


2/2 [==============================] - 1s 327ms/step - loss: 1.7873 - accuracy: 0.3019 - val_loss: 1.7861 - val_accuracy: 0.2857
Epoch 2/10
2/2 [==============================] - 0s 21ms/step - loss: 1.7679 - accuracy: 0.3019 - val_loss: 1.7775 - val_accuracy: 0.2857
Epoch 3/10
2/2 [==============================] - 0s 20ms/step - loss: 1.7512 - accuracy: 0.3019 - val_loss: 1.7690 - val_accuracy: 0.2857
Epoch 4/10
2/2 [==============================] - 0s 19ms/step - loss: 1.7359 - accuracy: 0.3019 - val_loss: 1.7624 - val_accuracy: 0.2857
Epoch 5/10
2/2 [==============================] - 0s 20ms/step - loss: 1.7178 - accuracy: 0.3019 - val_loss: 1.7552 - val_accuracy: 0.2857
Epoch 6/10
2/2 [==============================] - 0s 20ms/step - loss: 1.7059 - accuracy: 0.3019 - val_loss: 1.7488 - val_accuracy: 0.2857
Epoch 7/10
2/2 [==============================] - 0s 19ms/step - loss: 1.6861 - accuracy: 0.3019 - val_loss: 1.7411 - val_accuracy: 0.2857
Epoch 8/10
2/2 [=====================

In [13]:
# Assuming your test data is stored in X_test and y_test
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')


1/1 [==============================] - 0s 20ms/step - loss: 1.6855 - accuracy: 0.2353
Test loss: 1.685490369796753, Test accuracy: 0.23529411852359772


In [14]:
from konlpy.tag import Komoran
komoran=Komoran()
import json
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd

tokenizer = Tokenizer()
#tokenizer.fit_on_texts(sentences)
#word_index = tokenizer.word_index
#sequences = tokenizer.texts_to_sequences(sentences)
#one_hot_encoded = to_categorical(sequences)


In [15]:
len(list(word_index.keys()))

50

In [24]:
def preprocess_text(complaint, word_index, max_sequence_length, tokenizer, df):
    # Create a Tokenizer and fit on the sentences
    tokenizer = Tokenizer()


    data = []
    for s in df['NLP'].iloc[:] :
        data.append(s)

    tokenizer.fit_on_texts(data)

    tokenized_complaint = komoran.morphs(complaint)  # Implement your tokenization function

    li = []

    for i in tokenized_complaint :
        if i in list(word_index.keys()) :
            li.append(word_index[i])
        else : 
            li.append(len(list(word_index.keys())) + 1)


    max_sequence_length = 10
    padded_sequences = pad_sequences([li], maxlen=max_sequence_length, padding='post')

    print(padded_sequences)
    one_hot_encoded = to_categorical(padded_sequences, num_classes=len(word_index) + 1)

    return one_hot_encoded

def map_index_to_category(predicted_index):
    categories = {0: '인사', 1: '작별인사', 2: '대건고 위치',
                  3: '대건고 교훈', 4: '대건고 교목', 5: '대건고 교화'}

    return categories[predicted_index]

def predict_category(model, complaint, word_index, max_sequence_length, tokenizer, df):
    # Preprocess the complaint
    X_new_data = preprocess_text(complaint, word_index, max_sequence_length, tokenizer, df)

    # Make predictions using the model
    predictions = model.predict(X_new_data)
    
    print(X_new_data)

    # Get the index of the category with the highest probability
    predicted_classes = predictions.argmax(axis=-1)

    print(predicted_classes)
    
    category = map_index_to_category(predicted_classes[0])

    return category

In [26]:
with open('Word_Dict.pkl','rb') as f:
   word_index = pickle.load(f)

user_complaint = "대건고등학교의 교목이 뭐야?"
max_sequence_length = 10

category = predict_category(model, user_complaint, word_index, max_sequence_length, tokenizer, df)
category



[[ 3  1  8  7 11 14  2  0  0  0]]
1/1 [==============================] - 0s 15ms/step
[[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 

'대건고 교화'

In [18]:
#compiled_chatbot_model.save('Chatbot_Model.h5')

In [19]:
import tensorflow as tf

# Load the trained model
#model = tf.keras.models.load_model('Chatbot_Model.h5')


In [20]:
X_train.shape

(67, 10, 51)